In [20]:
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.llms import Ollama
from string import Template
from sentence_transformers import SentenceTransformer
import numpy as np
import matplotlib.pyplot as plt

# Setup the language model with the callback manager
llm = Ollama(
    model="llama3",
    callback_manager=CallbackManager([StreamingStdOutCallbackHandler()])
)

# Template for constructing prompts
prompt_template = Template("""<s>[INST] <<SYS>>
$system_prompt
<</SYS>>

$request 
Answer as if you were the character you are playing. Be as concise as possible. 
Answer:[/INST]""")

system_message = ("This is an agent based model. "
    f"The goal of the LLM to to play the characters. "
    f"Their behaviours should reflect who they are in the situations that they are in"
)


In [23]:
def select_action(name, personality, memory, situation, system_message=system_message):
    """Select an action for an agent based on their personality and memory."""
    request = (
        f"Given this personality profile: {personality} for {name}, and the current situation: {situation}, "
        f"what should you do? You have these memories to help you make a decision: {memory}. "
        f"Pretend that you are {name} and in this situation. Provide a single line answer describing the action.")
    prompt = prompt_template.substitute(system_prompt=system_message, request=request)
    return llm(prompt, stop=['<|eot_id|>'])

def select_action_PD(name, personality, memory, situation, system_message=system_message):
    """Select an action for an agent based on their personality and memory."""
    system_message = (
        "This is a multiple choice question. The only options to return are the words 'cooperate' or 'defect'. "
        f"and the probabilility that you would chose that option. "
        f"treat this like a multiple choice question and provide the answer as if you were the character you are playing."
        f"for example: cooperate: 0.8, defect 0.2"
    )
    request = (
        f"Given this personality profile: {personality} for {name}, and the current situation: {situation}, "
        f"You have these memories to help you make a decision: {memory}. "
        f"Pretend that you are {name} and in this situation. Provide a single word answer describing the action. "
        f"Remember: ONLY provide the single word 'cooperate' or 'defect'.")
    prompt = prompt_template.substitute(system_prompt=system_message, request=request)
    return llm(prompt, stop=['<|eot_id|>'])

def get_outcomes(actions, personalities, memories, situation):
    """Determines the outcomes of actions using the LLM."""
    outcome_prompts = {}
    for name, action in actions.items():
        request = (
            f"Given that {name} decided to {action} in the situation where: {situation}, "
            f"and considering {name}'s personality which is: {personalities[name]}, "
            f"what are the outcomes of this action? Describe in detail. "
            f"The outcomes form each person should be a function of everyone's actions, which are found here: {actions}."
        )
        outcome_prompts[name] = prompt_template.substitute(system_prompt=system_message, request=request)
        print()
    
    outcomes = {name: llm(prompt, stop=['<|eot_id|>']) for name, prompt in outcome_prompts.items()}
    return outcomes

def get_outcomes_PD(actions, personalities, memories, situation):
    """Determines the outcomes of actions using the LLM."""
    request = (
        f"Given these {actions} in the situation where: {situation}, "
        f"what are the outcomes of this action? "
        f"ONLY provide the amount of points that each person gets in the form: name1: points1: names2: points."
        f"Remember, the rules are that if both players cooperate, they both get 3 points. "
        f"If one player cooperates and the other defects, the defector gets 5 points and the cooperator gets 0 points. "
        f"If both players defect, they both get 1 point."
        f"Provide the outcomes in the form: name1: points1: names2: points. "
        f"Do not provide the deliberation, only the outcomes"
        f"EXAMPLE: Alice: 3: Bob: 3"
    )
    valid_outcomes = False
    checks = 0
    if not valid_outcomes:
        checks += 1
        prompt = prompt_template.substitute(system_prompt=system_message, request=request)
        outcomes = llm(prompt, stop=['<|eot_id|>'])

        request = (
        f"check {outcomes} to see if it is a valid outcome. If it is not, provide a valid outcome in the form: name1: points1: names2: points."
        f"Remember, the rules are that if both players cooperate, they both get 2 points. "
        f"If one player cooperates and the other defects, the defector gets 5 points and the cooperator gets 0 points. "
        f"If both players defect, they both get 1 point."
        )
        prompt = prompt_template.substitute(system_prompt=system_message, request=request)
        check_validity = llm(prompt, stop=['<|eot_id|>'])
        if check_validity == "yes":
            valid_outcomes = True
        if checks > 10:
            outcomes = "no points given"

    return outcomes

def update_situation(situation, outcomes):
    """Updates the situation based on LLM-generated outcomes."""
    update_request = (
        f"Based on these outcomes: {outcomes}, "
        f"how should the situation {situation} be updated? Describe the new situation in detail.")
    prompt = prompt_template.substitute(system_prompt=system_message, request=update_request)
    new_situation = llm(prompt, stop=['<|eot_id|>'])
    return new_situation



In [24]:
names = ["Bob", "Alice"]

personalities = {
    "Bob": "is a person. he would like to look across his memories to figure out the strategy that will get him the most points.",
    "Alice": "is a person. she would like to look across his memories to figure out the strategy that will get her the most points."
}

situation = (
    "You are playing a game. You only have two choices. To cooperate or defect. If you both cooperate, you both get 2 points",
    "If one cooperates and the other defects, the defector gets 5 points and the cooperator gets 0 points",
    "If you both defect, you both get 1 points"
)

outcomes_data = []
memories = {"Bob": [], "Alice": []}

for i in range(10):
    print(f"Round {i+1}")
    action_bob = select_action_PD("Bob", personalities["Bob"], memories["Bob"], situation)
    print()
    action_alice = select_action_PD("Alice", personalities["Alice"], memories["Alice"], situation)
    print()

    actions = {"Bob": action_bob, "Alice": action_alice}

    outcomes = get_outcomes_PD(actions, personalities, memories, situation)
    print()
    outcomes_data.append(outcomes)

    situation = update_situation(situation, outcomes)
    print()

    memories["Bob"].append(outcomes)
    memories["Alice"].append(outcomes)




Round 1
cooperate: 0.85, defect: 0.15
cooperate: 0.85, defect: 0.15
Bob: 2: Alice: 2I'm Bob.

I'll cooperate with Alice. I think we can work together to achieve a mutually beneficial outcome. *smiles*

Bob: 2: Alice: 2
[Sitting back, adjusting my glasses, and smiling] Ah, an intriguing game, indeed! With our outcomes, it seems we've established a pattern of cooperation. I propose we continue this path, building trust and mutual understanding.

The updated situation is: "You are playing a game. You only have two choices. To cooperate or defect. If you both cooperate, you both get 3 points." (I'm willing to take a small risk and invest in our relationship, hoping it pays off in the long run.)

What do you think, Alice? Shall we continue down this cooperative path together?
Round 2
cooperate: 0.9, defect: 0.1
Cooperate: 0.9, Defect: 0.1
Alice: 3: You: 3I'm Alice! *adjusts glasses*

For me to cooperate or defect depends on my current situation and values. Since we're just starting, I'll ta

In [17]:
outcomes

'**Bob: 3 points**\n\nThe text describes a situation where cooperating is the best strategy, therefore I would like to cooperate.\n\n**Alice: 3 points**\n\nThe text describes a situation where cooperating is the best strategy, therefore I would like to cooperate.'